# Data transformation

In [1]:
import pandas as pd
import seaborn as sns
from google.colab import files
sns.set()

In [2]:
uploaded = files.upload()

Saving AmesHousing.csv to AmesHousing.csv


In [3]:
ames_housing = pd.read_csv("AmesHousing.csv")
ames_housing.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,...,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


> Should any of the cells above fail, contact the maintainers of scikit-zero ;)

Peeking the first rows we can see that this is a very large dataset w.r.t. number of features. Let's check how many samples it has:

In [4]:
ames_housing.shape

(2930, 82)

> Kinda like iris, right? 🤣

Let's start isolating the feature we would like to predict from the input features:

In [5]:
X = ames_housing.drop(["SalePrice"], axis=1)
y = ames_housing["SalePrice"]

> Note that the feature we want to predict is real-valued, so we have a **regression** problem.

To make our analysis a bit more practical for the purposes of this notebook, we'll **filter out** features according to some pre-defined criteria:
* features that are clearly unrelated to the target feature;
* features that present more than 90% of missing values;
* features with low variance:
    * Categorical features: the most ocurring value is in more than 90% of the samples.
    * Numerical features: the 90% quantile matches the median.
    
> Note that this filtering is specific to this notebook and could likely represent a premature decision in the machine learning modeling process. We do it here to make the dataset a bit simpler. For this reason also, we pre-identified these features rather than computing them in the notebook.

In [6]:
unrelated = ["Order", "PID", "MS SubClass"]
many_missing = ["Pool QC", "Misc Feature", "Alley"]
low_variance_nominal = ['Paved Drive', 'Electrical', 'Functional', 'Central Air', 'Land Slope', 'Heating', 'Roof Matl', 'Condition 2']
low_variance_ordinal = ['Garage Cond', 'Street', 'Utilities']
low_variance_numerical = ["Low Qual Fin SF", "Bsmt Half Bath", "Full Bath", "Kitchen AbvGr",
                          "Fireplaces", "3Ssn Porch", "Screen Porch", "Pool Area", "Misc Val"]

In [7]:
X = X.drop(unrelated + many_missing +
           low_variance_nominal + low_variance_ordinal,
           axis=1)

> Even with this filtering, this dataset is still quite complex. In real life, we would never go for modelling before having conducted an exploratory data analysis. Here, we are making things a bit simplistic since we're skipping that, but bear with me :)

In [8]:
X.shape

(2930, 64)

## Encoding ordinal features

To properly prepare the data for modelling, the first thing we need to do is isolate feature types. This is very important, since different types of feature demand different preparation techniques. From a quick look at the data, I have isolated the following three feature subsets:

> If you don't understand the difference between feature types, check [pandas-zero](https://github.com/leobezerra/pandas-zero). 

> If you do, notice that a few ordinal features are being listed as nominal. The reason is that we do not have a rich data dictionary available, and for those features it was not straightforward to understand the ranking implied by the values.

In [9]:
nominal = ["MS Zoning", "Lot Shape", "Land Contour",  "Lot Config", 
           "Neighborhood", "Condition 1", "Bldg Type", "House Style", 
           "Roof Style", "Exterior 1st", "Exterior 2nd", "Mas Vnr Type", 
           "Foundation", "BsmtFin Type 1", "BsmtFin Type 2", "Garage Type", 
           "Fence", "Sale Type", "Sale Condition", "Bsmt Exposure", "Garage Finish"]

ordinal = ["Exter Qual", "Exter Cond", "Bsmt Qual", "Bsmt Cond",  "Heating QC",
           "Kitchen Qual", "Fireplace Qu", "Garage Qual"]

numerical = ["Lot Frontage", "Lot Area", "Year Built", "Year Remod/Add", "Mas Vnr Area",
             "BsmtFin SF 1", "BsmtFin SF 2", "Bsmt Unf SF", "Total Bsmt SF", "1st Flr SF",
             "2nd Flr SF", "Low Qual Fin SF", "Gr Liv Area", "Bsmt Full Bath", "Bsmt Half Bath", 
             "Full Bath", "Half Bath", "Bedroom AbvGr", "Kitchen AbvGr", "TotRms AbvGrd",
             "Fireplaces", "Garage Yr Blt", "Garage Cars", "Garage Area", "Wood Deck SF",
             "Open Porch SF", "Enclosed Porch", "3Ssn Porch", "Screen Porch", "Pool Area",
             "Misc Val", "Mo Sold", "Yr Sold", "Overall Cond", "Overall Qual"]

Let's temporarily isolate the ordinal features to see how we can prepare them:

In [10]:
X_ordinal = X[ordinal]
X_ordinal.head()

,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Heating QC,Kitchen Qual,Fireplace Qu,Garage Qual
0,TA,TA,TA,Gd,Fa,TA,Gd,TA
1,TA,TA,TA,TA,TA,TA,NaN,TA
2,TA,TA,TA,TA,TA,Gd,NaN,TA
3,Gd,TA,TA,TA,Ex,Ex,TA,TA
4,TA,TA,Gd,TA,Gd,TA,TA,TA


Checking these features a bit we see that they follow a ranking pattern:
0. `TA`: to be assessed
1. `Po`: poor
2. `Fa`: fair
3. `Gd`: good
4. `Ex`: excellent

The encoding we apply to ordinal features is converting each ranking category into an integer, so that higher ranks are represented by larger values. 

> `TA` is equivalent to a missing value, and we'll rank it 0. Again, this could be premature, but it's a reasonable choice here.

Encoding ordinal features in scikit-learn can be done with the `OrdinalEncoder` preprocessor from the `preprocessing` module:

In [12]:
from sklearn.preprocessing import OrdinalEncoder

`OrdinalEncoder` takes as input a list of lists: 
- one list per feature
- each list should indicate the ranking of the categories

For our example, the same list can be used to indicate the ranking of the categories:

In [13]:
conditions = ["TA", "Po", "Fa", "Gd", "Ex"]

One limitation of `OrdinalEncoder` is that we cannot indicate that the same list will be used for multiple features. To replicate the list as many times as the number of features we have, we're gonna use the list comprehension notation we've discussed back in pandas-zero:

> the `for i in range(n_features)` means we will have `n_features` repetitions of `conditions`

In [14]:
n_features = X_ordinal.shape[1]
condition_lists = [conditions for i in range(n_features)]

We can now create our encoder for ordinal features:

In [15]:
condition_encoder = OrdinalEncoder(categories=condition_lists)

Another limitation of `OrdinalEncoder` is that it cannot be applied to data with missing values. For this reason, we're gonna use a pipeline where we first impute `'TA'` to the missing values, and then apply the encoding:

In [16]:
from sklearn.impute import SimpleImputer
condition_imputer = SimpleImputer(strategy="constant", fill_value="TA")

In [17]:
from sklearn.pipeline import make_pipeline
ordinal_pipe = make_pipeline(condition_imputer, condition_encoder)

When we use a pipeline solely for data preparation, we use `fit()` and `transform()`, rather than `fit()` and `predict()`. In this case, we'll make it simpler and use `fit_transform()` directly:

> Note that we wrap the output of the pipeline in a `DataFrame` object and preserve the column values.

In [18]:
X_ordinal = pd.DataFrame(ordinal_pipe.fit_transform(X_ordinal), columns=X_ordinal.columns)
X_ordinal.head()

,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Heating QC,Kitchen Qual,Fireplace Qu,Garage Qual
0,0.0,0.0,0.0,3.0,2.0,0.0,3.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
3,3.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0
4,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0


Now our ordinal features have been encoded properly, and we can move on to nominal features.

## Encoding nominal features

Once again, let's start isolating our nominal features:

In [19]:
X_nominal = X[nominal]
X_nominal.head()

,MS Zoning,Lot Shape,Land Contour,Lot Config,Neighborhood,Condition 1,Bldg Type,House Style,Roof Style,Exterior 1st,Exterior 2nd,Mas Vnr Type,Foundation,BsmtFin Type 1,BsmtFin Type 2,Garage Type,Fence,Sale Type,Sale Condition,Bsmt Exposure,Garage Finish
0,RL,IR1,Lvl,Corner,NAmes,Norm,1Fam,1Story,Hip,BrkFace,Plywood,Stone,CBlock,BLQ,Unf,Attchd,NaN,WD,Normal,Gd,Fin
1,RH,Reg,Lvl,Inside,NAmes,Feedr,1Fam,1Story,Gable,VinylSd,VinylSd,None,CBlock,Rec,LwQ,Attchd,MnPrv,WD,Normal,No,Unf
2,RL,IR1,Lvl,Corner,NAmes,Norm,1Fam,1Story,Hip,Wd Sdng,Wd Sdng,BrkFace,CBlock,ALQ,Unf,Attchd,NaN,WD,Normal,No,Unf
3,RL,Reg,Lvl,Corner,NAmes,Norm,1Fam,1Story,Hip,BrkFace,BrkFace,None,CBlock,ALQ,Unf,Attchd,NaN,WD,Normal,No,Fin
4,RL,IR1,Lvl,Inside,Gilbert,Norm,1Fam,2Story,Gable,VinylSd,VinylSd,None,PConc,GLQ,Unf,Attchd,MnPrv,WD,Normal,No,Fin


In [20]:
X_nominal.shape

(2930, 21)

The most common approach to nominal feature encoding is called **one-hot encoding**. Let's take feature `Lot Shape` as example:

In [21]:
X_nominal["Lot Shape"].value_counts(dropna=False)

Reg    1859
IR1     979
IR2      76
IR3      16
Name: Lot Shape, dtype: int64

With one-hot encoding, feature `Lot Shape` gets replaced by binary features, one for each of the possible values presented by `Lot Shape`. Let's start importing the scikit-learn resources we're gonna need:

In [22]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

The `make_column_transformer` from the `compose` module allows us to specify encodings for specific features.

- Let's understand the code below:
```python 
nominal_encoder = make_column_transformer(
                                            (OneHotEncoder(sparse=False), ["Lot Shape"]),
                                            remainder="drop")
```
> The `sparse=False` attribute indicates that the output should be a regular numpy array, rather than a more efficient scipy sparse matrix. Even if the former is less efficient, it is more readable and helps to keep the notebook understandable.

- Creates the column transformer, specifying that a `OneHotEncoder()` should be applied to `["Lot Shape"]`, and that the remainder of the features should be dropped. 
```python 
encoded_data = nominal_encoder.fit_transform(X_nominal)
```
- Applies the `fit_transform()` method to `X_nominal`. 
```python
encoded_data = pd.DataFrame(encoded_data,
                              columns=nominal_encoder.get_feature_names())
```
- Wrap the numpy array as a DataFrame, retrieving the feature names created by the encoder.

Let's see the output of that:

In [23]:
nominal_encoder = make_column_transformer(
                                          (OneHotEncoder(sparse=False), ["Lot Shape"]),
                                          remainder="drop")
encoded_data = nominal_encoder.fit_transform(X_nominal)
encoded_data = pd.DataFrame(encoded_data,
                            columns=nominal_encoder.get_feature_names())
encoded_data.head()

,onehotencoder__x0_IR1,onehotencoder__x0_IR2,onehotencoder__x0_IR3,onehotencoder__x0_Reg
0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0


Note that the features are now labeled as a function of the feature values. For each sample, only one of the enconded feature values equals 1.0, since that was the original value before the encoding.

> You can imagine that this type of encoding explodes the number of features in a dataframe, right? Let's see by how much!

In [24]:
nominal_pipe = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(sparse=False))
pd.DataFrame(nominal_pipe.fit_transform(X_nominal),
             columns=nominal_pipe.steps[1][1].get_feature_names(nominal))

,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Lot Shape_IR1,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,...,BsmtFin Type 1_Unf,BsmtFin Type 2_ALQ,BsmtFin Type 2_BLQ,BsmtFin Type 2_GLQ,BsmtFin Type 2_LwQ,BsmtFin Type 2_Rec,BsmtFin Type 2_Unf,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Fence_GdPrv,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial,Bsmt Exposure_Av,Bsmt Exposure_Gd,Bsmt Exposure_Mn,Bsmt Exposure_No,Garage Finish_Fin,Garage Finish_RFn,Garage Finish_Unf
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2925,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2926,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,

> Note that we could have used better strategies than a `SimpleImputer(strategy='most_frequent')`, but that would make the notebook even longer 🙃

## Preparing numerical features

When dealing with numerical features, our major concerns regard the data scale and distribution. Scikit-learn offers several resources to address these issues, as we'll see below.

### Feature scaling

A very important aspect in data preprocessing concerns feature scaling. Take an algorithm that considers Euclidean distances, such as kNN. If a feature has much larger values than another, the relative importance of these features will be unbalanced. Sometimes, this comes in your favor, since the features with larger values are indeed the most important. Other times, your model will be undervaluing the most relevant features.

Several scaling techniques can be used to make different features present the same scale, but they only make sense when we're talking about numerical features. For now, let's isolate the numerical features from the remaining. 

In [25]:
X_num = X[numerical]
X_num.head()

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,141.0,31770,1960,1960,112.0,639.0,0.0,441.0,1080.0,1656,0,0,1656,1.0,0.0,1,0,3,1,7,2,1960.0,2.0,528.0,210,62,0,0,0,0,0,5,2010,5,6
1,80.0,11622,1961,1961,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1961.0,1.0,730.0,140,0,0,0,120,0,0,6,2010,6,5
2,81.0,14267,1958,1958,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1958.0,1.0,312.0,393,36,0,0,0,0,12500,6,2010,6,6
3,93.0,11160,1968,1968,0.0,1065.0,0.0,1045.0,2110.0,2110,0,0,2110,1.0,0.0,2,1,3,1,8,2,1968.0,2.0,522.0,0,0,0,0,0,0,0,4,2010,5,7
4,74.0,13830,1997,1998,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,1997.0,2.0,482.0,212,34,0,0,0,0,0,3,2010,5,5


In [ ]:
X_num.shape

Note that now we're working with 35 features. Let's move on to the techniques so we can understand our options.

#### Fitting to a range

The first thing we notice about the numerical features of this dataset is that feature ranges are very different. Let's check, for instance, the range differences between features `Overall Qual`, `Year Built`, and `Gr Liv Area`:

In [26]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

,Overall Qual,Year Built,Gr Liv Area
count,2930.000000,2930.000000,2930.000000
mean,6.094881,1971.356314,1499.690444
std,1.411026,30.245361,505.508887
min,1.000000,1872.000000,334.000000
25%,5.000000,1954.000000,1126.000000
50%,6.000000,1973.000000,1442.000000
75%,7.000000,2001.000000,1742.750000
max,10.000000,2010.000000,5642.000000


We can scale all features to the `[0,1]` range using the `MinMaxScaler` preprocessor:

In [27]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
X_num = pd.DataFrame(minmax.fit_transform(X_num), columns=X_num.columns)
X_num.head()

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,0.410959,0.142420,0.637681,0.166667,0.0700,0.113218,0.000000,0.188784,0.176759,0.277673,0.000000,0.0,0.249058,0.333333,0.0,0.25,0.0,0.375,0.333333,0.384615,0.50,0.208333,0.4,0.354839,0.147472,0.083558,0.0,0.0,0.000000,0.0,0.000000,0.363636,1.0,0.500,0.555556
1,0.202055,0.048246,0.644928,0.183333,0.0000,0.082920,0.094364,0.115582,0.144354,0.118042,0.000000,0.0,0.105878,0.000000,0.0,0.25,0.0,0.250,0.333333,0.230769,0.00,0.211538,0.2,0.490591,0.098315,0.000000,0.0,0.0,0.208333,0.0,0.000000,0.454545,1.0,0.625,0.444444
2,0.205479,0.060609,0.623188,0.133333,0.0675,0.163536,0.000000,0.173801,0.217512,0.208990,0.000000,0.0,0.187453,0.000000,0.0,0.25,0.5,0.375,0.333333,0.307692,0.00,0.201923,0.2,0.209677,0.275983,0.048518,0.0,0.0,0.000000,0.0,0.735294,0.454545,1.0,0.625,0.555556
3,0.246575,0.046087,0.695652,0.300000,0.0000,0.188696,0.000000,0.447346,0.345336,0.373031,0.000000,0.0,0.334589,0.333333,0.0,0.50,0.5,0.375,0.333333,0.461538,0.50,0.233974,0.4,0.350806,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.272727,1.0,0.500,0.666667
4,0.181507,0.058566,0.905797,0.800000,0.0000,0.140149,0.000000,0.058647,0.151882,0.124764,0.339467,0.0,0.243971,0.000000,0.0,0.50,0.5,0.375,0.333333,0.307692,0.25,0.326923,0.4,0.323925,0.148876,0.045822,0.0,0.0,0.000000,0.0,0.000000,0.181818,1.0,0.500,0.444444


Note that now the min and max values for each feature now match:

In [28]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

,Overall Qual,Year Built,Gr Liv Area
count,2930.000000,2930.000000,2930.000000
mean,0.566098,0.719973,0.219610
std,0.156781,0.219169,0.095235
min,0.000000,0.000000,0.000000
25%,0.444444,0.594203,0.149209
50%,0.555556,0.731884,0.208742
75%,0.666667,0.934783,0.265401
max,1.000000,1.000000,1.000000


> Note how the range scaling made the means of `Year Built` and `Gr Liv Area` different, when they were originally very close!

#### Centering the data

Another transformation commonly applied to numerical data is centering the data so it presents zero mean. We can do this transformation with the `StandardScaler` preprocessor, following the same pattern adopted above:

In [29]:
from sklearn.preprocessing import StandardScaler
zero_mean = StandardScaler(with_std=False)
X_num = pd.DataFrame(zero_mean.fit_transform(X_num), columns=X_num.columns)
X_num.head()

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,0.245806,0.101064,-0.082292,-0.404443,0.006314,0.034793,-0.032584,-0.050626,0.004646,0.104273,-0.162448,-0.004395,0.029448,0.189549,-0.030567,-0.141638,-0.189761,0.018217,-0.01479,0.042846,0.350171,-0.058117,0.046637,0.037084,0.081635,0.019497,-0.022739,-0.005103,-0.027781,-0.002804,-0.002979,-0.110549,0.552389,-0.070392,-0.010542
1,0.036902,0.006890,-0.075046,-0.387776,-0.063686,0.004495,0.061781,-0.123828,-0.027760,-0.055358,-0.162448,-0.004395,-0.113732,-0.143784,-0.030567,-0.141638,-0.189761,-0.106783,-0.01479,-0.111000,-0.149829,-0.054912,-0.153363,0.172836,0.032478,-0.064061,-0.022739,-0.005103,0.180552,-0.002804,-0.002979,-0.019640,0.552389,0.054608,-0.121653
2,0.040327,0.019253,-0.096785,-0.437776,0.003814,0.085112,-0.032584,-0.065609,0.045399,0.035590,-0.162448,-0.004395,-0.032157,-0.143784,-0.030567,-0.141638,0.310239,0.018217,-0.01479,-0.034077,-0.149829,-0.064527,-0.153363,-0.108078,0.210146,-0.015544,-0.022739,-0.005103,-0.027781,-0.002804,0.732316,-0.019640,0.552389,0.054608,-0.010542
3,0.081423,0.004731,-0.024321,-0.271109,-0.063686,0.110271,-0.032584,0.207936,0.173222,0.199631,-0.162448,-0.004395,0.114979,0.189549,-0.030567,0.108362,0.310239,0.018217,-0.01479,0.119769,0.350171,-0.032476,0.046637,0.033051,-0.065837,-0.064061,-0.022739,-0.005103,-0.027781,-0.002804,-0.002979,-0.201458,0.552389,-0.070392,0.100569
4,0.016354,0.017210,0.185824,0.228891,-0.063686,0.061724,-0.032584,-0.180763,-0.020232,-0.048636,0.177019,-0.004395,0.024361,-0.143784,-0.030567,0.108362,0.310239,0.018217,-0.01479,-0.034077,0.100171,0.060473,0.046637,0.006170,0.083039,-0.018239,-0.022739,-0.005103,-0.027781,-0.002804,-0.002979,-0.292367,0.552389,-0.070392,-0.121653


> The parameter `with_std` defines whether `StandardScaler` also scales the variance. Here we disabled it not to spoil the next topic 😅

After the transformation, all features present zero mean:

In [30]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

,Overall Qual,Year Built,Gr Liv Area
count,2.930000e+03,2.930000e+03,2.930000e+03
mean,6.970154e-17,-8.813579e-17,8.127742e-18
std,1.567807e-01,2.191693e-01,9.523528e-02
min,-5.660978e-01,-7.199733e-01,-2.196101e-01
25%,-1.216534e-01,-1.257704e-01,-7.040136e-02
50%,-1.054228e-02,1.191077e-02,-1.086858e-02
75%,1.005688e-01,2.148093e-01,4.579117e-02
max,4.339022e-01,2.800267e-01,7.803899e-01


> Computers have a big trouble matching exact numbers when dealing with real values. The values shown in the mean column above are small enough to be considered zero by most algorithms.

#### Scaling the variance

A final transformation that is often applied to numerical data is scaling the variance of the data to one. 

> It is so common that the `StandardScaler` preprocessor does it by default! Since we're doing it step-wisely, we will tell the preprocessor it doesn't need to center the data again (`with_mean=False`).

In [31]:
unit_variance = StandardScaler(with_mean=False)
X_num = pd.DataFrame(unit_variance.fit_transform(X_num), columns=X_num.columns)
X_num.head()

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,3.072506,2.744381,-0.375537,-1.163488,0.056417,0.431097,-0.293973,-0.269134,0.064433,1.267003,-0.783185,-0.101005,0.309265,1.083694,-0.249311,-1.024793,-0.755203,0.176094,-0.207291,0.354167,2.162180,-0.710413,0.306647,0.256641,0.920121,0.214409,-0.358838,-0.103134,-0.285354,-0.063031,-0.089422,-0.448057,1.678499,-0.506718,-0.067254
1,0.461265,0.187097,-0.342468,-1.115542,-0.568996,0.055696,0.557395,-0.658284,-0.385015,-0.672643,-0.783185,-0.101005,-1.194427,-0.822046,-0.249311,-1.024793,-0.755203,-1.032234,-0.207291,-0.917535,-0.925143,-0.671234,-1.008387,1.196133,0.366061,-0.704493,-0.358838,-0.103134,1.854530,-0.063031,-0.089422,-0.079602,1.678499,0.393091,-0.776079
2,0.504073,0.522814,-0.441674,-1.259380,0.034081,1.054570,-0.293973,-0.348784,0.629649,0.432445,-0.783185,-0.101005,-0.337718,-0.822046,-0.249311,-1.024793,1.234675,0.176094,-0.207291,-0.281684,-0.925143,-0.788771,-1.008387,-0.747965,2.368594,-0.170937,-0.358838,-0.103134,-0.285354,-0.063031,21.985725,-0.079602,1.678499,0.393091,-0.067254
3,1.017759,0.128458,-0.110988,-0.779919,-0.568996,1.366306,-0.293973,1.105408,2.402474,2.425687,-0.783185,-0.101005,1.207523,1.083694,-0.249311,0.784028,1.234675,0.176094,-0.207291,0.990018,2.162180,-0.396980,0.306647,0.228735,-0.742060,-0.704493,-0.358838,-0.103134,-0.285354,-0.063031,-0.089422,-0.816513,1.678499,-0.506718,0.641571
4,0.204422,0.467348,0.848000,0.658466,-0.568996,0.764787,-0.293973,-0.960956,-0.280598,-0.590974,0.853432,-0.101005,0.255844,-0.822046,-0.249311,0.784028,1.234675,0.176094,-0.207291,-0.281684,0.618518,0.739214,0.306647,0.042697,0.935952,-0.200579,-0.358838,-0.103134,-0.285354,-0.063031,-0.089422,-1.184969,1.678499,-0.506718,-0.776079


After the transformation, all features present unit variance:

> And, by definition, unit standard deviation!

In [32]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].var()

Overall Qual    1.000341
Year Built      1.000341
Gr Liv Area     1.000341
dtype: float64

In [33]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

,Overall Qual,Year Built,Gr Liv Area
count,2.930000e+03,2.930000e+03,2.930000e+03
mean,9.806717e-16,-1.566816e-16,1.610771e-16
std,1.000171e+00,1.000171e+00,1.000171e+00
min,-3.611379e+00,-3.285571e+00,-2.306368e+00
25%,-7.760788e-01,-5.739484e-01,-7.393623e-01
50%,-6.725371e-02,5.435434e-02,-1.141430e-01
75%,6.415714e-01,9.802742e-01,4.809036e-01
max,2.768047e+00,1.277891e+00,8.195734e+00


#### Using pipelines

As discussed in the first steps with scikit-learn, pipelines are the best way to combine all the steps in a machine learning experiment. Here, we're gonna use them to encapsulate all the transformations applied to numerical features:

In [34]:
from sklearn.impute import KNNImputer
num_pipe = make_pipeline(KNNImputer(), MinMaxScaler(), StandardScaler())

X_num = X[numerical]
X_num = pd.DataFrame(num_pipe.fit_transform(X_num), columns=X_num.columns)
X_num.head()

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,3.144741,2.744381,-0.375537,-1.163488,0.055111,0.431395,-0.293990,-0.269336,0.064568,1.267003,-0.783185,-0.101005,0.309265,1.08424,-0.24922,-1.024793,-0.755203,0.176094,-0.207291,0.354167,2.162180,-0.673659,0.306589,0.256574,0.920121,0.214409,-0.358838,-0.103134,-0.285354,-0.063031,-0.089422,-0.448057,1.678499,-0.506718,-0.067254
1,0.446548,0.187097,-0.342468,-1.115542,-0.571744,0.055970,0.557521,-0.658536,-0.384947,-0.672643,-0.783185,-0.101005,-1.194427,-0.82193,-0.24922,-1.024793,-0.755203,-1.032234,-0.207291,-0.917535,-0.925143,-0.634402,-1.008648,1.196210,0.366061,-0.704493,-0.358838,-0.103134,1.854530,-0.063031,-0.089422,-0.079602,1.678499,0.393091,-0.776079
2,0.490780,0.522814,-0.441674,-1.259380,0.032723,1.054907,-0.293990,-0.348997,0.629867,0.432445,-0.783185,-0.101005,-0.337718,-0.82193,-0.24922,-1.024793,1.234675,0.176094,-0.207291,-0.281684,-0.925143,-0.752172,-1.008648,-0.748186,2.368594,-0.170937,-0.358838,-0.103134,-0.285354,-0.063031,21.985725,-0.079602,1.678499,0.393091,-0.067254
3,1.021573,0.128458,-0.110988,-0.779919,-0.571744,1.366664,-0.293990,1.105385,2.402954,2.425687,-0.783185,-0.101005,1.207523,1.08424,-0.24922,0.784028,1.234675,0.176094,-0.207291,0.990018,2.162180,-0.359606,0.306589,0.228664,-0.742060,-0.704493,-0.358838,-0.103134,-0.285354,-0.063031,-0.089422,-0.816513,1.678499,-0.506718,0.641571
4,0.181152,0.467348,0.848000,0.658466,-0.571744,0.765106,-0.293990,-0.961248,-0.280514,-0.590974,0.853432,-0.101005,0.255844,-0.82193,-0.24922,0.784028,1.234675,0.176094,-0.207291,-0.281684,0.618518,0.778834,0.306589,0.042597,0.935952,-0.200579,-0.358838,-0.103134,-0.285354,-0.063031,-0.089422,-1.184969,1.678499,-0.506718,-0.776079


In [35]:
X_num[["Overall Qual", "Year Built", "Gr Liv Area"]].describe()

,Overall Qual,Year Built,Gr Liv Area
count,2.930000e+03,2.930000e+03,2.930000e+03
mean,9.806717e-16,-1.566816e-16,1.569469e-16
std,1.000171e+00,1.000171e+00,1.000171e+00
min,-3.611379e+00,-3.285571e+00,-2.306368e+00
25%,-7.760788e-01,-5.739484e-01,-7.393623e-01
50%,-6.725371e-02,5.435434e-02,-1.141430e-01
75%,6.415714e-01,9.802742e-01,4.809036e-01
max,2.768047e+00,1.277891e+00,8.195734e+00


### Transforming distributions

The scaling conducted above does not affect the distribution shape of the data. From many aspects, having the data distributed according to a normal distribution is desirable. 

> In pandas-zero, we discussed how to transform a logarithmic distribution into a normal distribution.

> We also discussed how to check if a distribution differs from a normal distribution. Tip: the second value in each parenthesis should be greater than or equal to 0.05.

In [36]:
from scipy.stats import normaltest
X_num.apply(normaltest)

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,1.073949e+03,5287.955695,2.098442e+02,81252.180747,1654.685532,9.821822e+02,2487.542063,3.231489e+02,9.256025e+02,1.014472e+03,3.133438e+02,5079.268155,7.757969e+02,3.506242e+02,2345.405533,8.383132e+01,9.714208e+02,1.738221e+02,2566.434475,2.937714e+02,2.175452e+02,1.591895e+02,2.937469e+01,8.198528e+01,1.199085e+03,1672.317747,2565.135491,4911.405733,2404.950984,5967.962542,6738.477261,6.080109e+01,1734.308579,2.386285e+02,17.880610
1,6.237926e-234,0.000000,2.709692e-46,0.000000,0.000000,5.270451e-214,0.000000,6.747012e-71,1.018534e-201,5.131162e-221,9.083538e-69,0.000000,3.450279e-169,7.293367e-77,0.000000,6.255479e-19,1.144617e-211,1.798954e-38,0.000000,1.615685e-64,5.763244e-48,2.706688e-35,4.181837e-07,1.574429e-18,4.187265e-261,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.269180e-14,0.000000,1.522235e-52,0.000131


Scikit-learn offers two preprocessing components to transform distributions into normal distributions, which we discuss next. 

#### Quantization

Quantization is the act of replacing a feature value for the quantile it belongs to. This is a non-parametric transformation that can be mapped to different distributions. scikit-learn provides the `QuantileTransformer` preprocessor, where we can configure the desired output distribution using the `output_distribution` argument:

In [37]:
from sklearn.preprocessing import QuantileTransformer
num_pipe = make_pipeline(KNNImputer(),
                         MinMaxScaler(),
                         StandardScaler(),
                         QuantileTransformer(output_distribution='normal'))

X_num = X[numerical]
X_num = pd.DataFrame(num_pipe.fit_transform(X_num), columns=X_num.columns)
X_num.head()

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,2.435512,2.293377,-0.418371,-0.835558,0.493553,0.483658,-5.199338,-0.051460,0.235509,1.236652,-5.199338,-5.199338,0.465405,0.235509,-5.199338,-0.741388,-5.199338,0.168912,-2.575483,0.171458,1.727466,-0.642543,-0.427976,0.353487,1.014571,0.570161,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-0.636387,5.199338,-0.41290,0.007527
1,0.616541,0.688795,-0.384375,-0.803958,-5.199338,0.165095,1.298271,-0.520661,-0.321611,-0.595437,-5.199338,-5.199338,-1.384628,-5.199338,-5.199338,-0.741388,-5.199338,-0.963423,-2.575483,-1.346386,-5.199338,-0.601439,-0.892038,1.215372,0.441766,-5.199338,-5.199338,-5.199338,1.460531,-5.199338,-5.199338,-0.042668,5.199338,0.31765,-1.278137
2,0.708752,1.276491,-0.499230,-0.912774,0.482248,1.031509,-5.199338,-0.139710,0.720036,0.551294,-5.199338,-5.199338,-0.249131,-5.199338,-5.199338,-0.741388,0.369558,0.168912,-2.575483,-0.226489,-5.199338,-0.731515,-0.892038,-0.711981,1.882160,0.147316,-5.199338,-5.199338,-5.199338,-5.199338,3.271258,-0.042668,5.199338,0.31765,0.007527
3,1.233962,0.574549,-0.160010,-0.582009,-5.199338,1.292822,-5.199338,1.071784,2.225955,2.119300,-5.199338,-5.199338,1.236652,0.235509,-5.199338,-0.109365,0.369558,0.168912,-2.575483,0.823172,1.727466,-0.348150,-0.427976,0.293986,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-0.791884,5.199338,-0.41290,0.615025
4,0.248427,1.194699,0.504922,0.247134,-5.199338,0.803369,-5.199338,-0.971082,-0.182928,-0.480839,0.663545,-5.199338,0.393598,-5.199338,-5.199338,-0.109365,0.369558,0.168912,-2.575483,-0.226489,0.016310,0.458426,-0.427976,0.046436,1.027247,0.118204,-5.199338,-5.199338,-5.199338,-5.199338,-5.199338,-1.140077,5.199338,-0.41290,-1.278137


In [38]:
X_num.apply(normaltest)

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,7.673138e+02,9.565326,2.909403e+01,7.158847e+02,12780.23447,48715.661562,1.298176e+03,9.776084e+02,9.739746e+02,10.358345,11877.349263,4103.114183,10.000162,12247.831917,2218.72553,9.712602e+02,3909.660107,3.616127e+02,2576.477074,18.041201,11569.466389,9.226258,1.330737e+03,1.080461e+03,11091.641068,11352.947635,9.449530e+02,4206.317077,1717.092186,5634.281474,2884.47501,3.224705e+02,0.208019,1.165202e+03,3.341217e+02
1,2.398480e-167,0.008374,4.811849e-07,3.528697e-156,0.00000,0.000000,1.272503e-282,5.188492e-213,3.192329e-212,0.005633,0.000000,0.000000,0.006737,0.000000,0.00000,1.240323e-211,0.000000,2.997769e-79,0.000000,0.000121,0.000000,0.009921,1.081768e-289,2.403639e-235,0.000000,0.000000,6.398333e-206,0.000000,0.000000,0.000000,0.00000,9.471374e-71,0.901217,9.542087e-254,2.795033e-73


From the results above, we see that only the `Yr Sold` feature is now matched with a normal distribution.

> Note, however, that the p-value (second value in each parenthesis) increases for most features, indicating that the transformed distributions are closer to a normal distribution than the original ones.

#### Power transformations

These parametric transformations are similar in spirit to the logarithmic transformation we discussed in pandas-zero. scikit-learn provides two options within the `PowerTransformer` preprocessor:

* the Yeo-Johnson transform, which works for positive and negative values;
* the Box-Cox transform, which only works for strictly positive values.

Since we have centered the data, we have to use the Yeo-Johnson transform (the default method for `PowerTransform`):

In [39]:
from sklearn.preprocessing import PowerTransformer
num_pipe = make_pipeline(KNNImputer(), 
                         MinMaxScaler(),
                         StandardScaler(),
                         PowerTransformer())

X_num = X[numerical]
X_num = pd.DataFrame(num_pipe.fit_transform(X_num), columns=X_num.columns)
X_num.head()

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,2.669279,2.541096,-0.537217,-1.159779,0.809950,0.670462,-0.368129,-0.081646,0.157076,1.258181,-0.842670,-0.117647,0.493615,1.151483,-0.252033,-1.027792,-0.767143,0.197861,-0.141622,0.472589,1.782616,-0.720307,0.258970,0.285649,1.182051,0.757070,-0.430381,-0.113091,-0.309413,-0.066758,-0.190872,-0.414845,1.616676,-0.452481,-0.022186
1,0.525415,0.577464,-0.507706,-1.128207,-0.767414,0.317560,2.720129,-0.617002,-0.323049,-0.657432,-0.842670,-0.117647,-1.451482,-0.842074,-0.252033,-1.027792,-0.767143,-1.040050,-0.141622,-0.956186,-0.988435,-0.684784,-1.007225,1.183096,0.812888,-0.993466,-0.430381,-0.113091,3.231985,-0.066758,-0.190872,-0.033869,1.616676,0.482078,-0.765380
2,0.566562,1.012291,-0.595214,-1.221698,0.778855,1.131166,-0.368129,-0.183596,0.693822,0.617919,-0.842670,-0.117647,-0.207813,-0.842074,-0.252033,-1.027792,1.288899,0.197861,-0.141622,-0.177517,-0.988435,-0.790804,-1.007225,-0.740079,1.728307,0.243186,-0.430381,-0.113091,-0.309413,-0.066758,5.243946,-0.033869,1.616676,0.482078,-0.022186
3,1.037903,0.488387,-0.290255,-0.894259,-0.767414,1.324253,-0.368129,1.145152,2.121218,1.952530,-0.842670,-0.117647,1.212083,1.151483,-0.252033,0.791276,1.288899,0.197861,-0.141622,1.017913,1.782616,-0.430446,0.258970,0.258217,-0.897997,-0.993466,-0.430381,-0.113091,-0.309413,-0.066758,-0.190872,-0.808867,1.616676,-0.452481,0.667296
4,0.270930,0.947835,0.864076,0.582298,-0.767414,0.931750,-0.368129,-1.095273,-0.207085,-0.540763,1.083004,-0.117647,0.443468,-0.842074,-0.252033,0.791276,1.288899,0.197861,-0.141622,-0.177517,0.767561,0.766894,0.258970,0.073811,1.190581,0.191563,-0.430381,-0.113091,-0.309413,-0.066758,-0.190872,-1.213567,1.616676,-0.452481,-0.765380


In [40]:
X_num.apply(normaltest)

,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,1.922360e+02,1.606552e+02,2194.074965,16805.948973,21305.429495,2231.836494,1.294142e+03,3.255885e+02,1.779196e+02,1.151028,13735.281306,4083.878564,1.665884,11979.975144,2190.301262,1.541414e+02,13358.019254,1.112270e+02,3971.709863,2.745239,26607.046209,5.463810e+02,21.145540,3.930828e+01,18007.053048,19381.912879,9.292967e+02,4186.144852,1697.103463,5603.937326,2867.054283,4.934362e+01,2295.255237,3.015079e+02,4.574009
1,1.805022e-42,1.300691e-35,0.000000,0.000000,0.000000,0.000000,9.562689e-282,1.992254e-71,2.318705e-39,0.562416,0.000000,0.000000,0.434768,0.000000,0.000000,3.377694e-34,0.000000,7.036451e-25,0.000000,0.253442,0.000000,2.264037e-119,0.000026,2.912827e-09,0.000000,0.000000,1.606181e-202,0.000000,0.000000,0.000000,0.000000,1.928274e-11,0.000000,3.375938e-66,0.101570


This time, `Yr Sold` does not match a normal distribution. Yet, a larger number of features do, namely `Overall Qual`, `1st Flr SF`, `Gr Liv Area`, and `TotRms AbvGrd`. 

## Wrapping it all

Although we should always deal with features according to their type, we can prepare our data without having to split our dataset as we did above:

In [41]:
full_pipe = make_column_transformer(
                                    (ordinal_pipe, ordinal),
                                    (nominal_pipe, nominal),
                                    (num_pipe, numerical)
                                    )

In order to access feature names produced by the process of transforming nominal features, we'll fit that pipeline to the data, even if we're not gonna use it to transform the data:

In [42]:
nominal_pipe.fit(X[nominal])
nominal_encoder = nominal_pipe.steps[1][1] # this accesses the OneHotEncoder within the pipeline
nominal_transformed = nominal_encoder.get_feature_names(nominal).tolist()

The feature names for the whole dataset will be the concatenation of the following features:

In [43]:
column_names = ordinal + nominal_transformed + numerical

We can now transform the data, preserving the dataframe feature names and therefore its readibility:

In [44]:
X_transformed = pd.DataFrame(full_pipe.fit_transform(X),
                             columns=column_names)
X_transformed.head()

,Exter Qual,Exter Cond,Bsmt Qual,Bsmt Cond,Heating QC,Kitchen Qual,Fireplace Qu,Garage Qual,MS Zoning_A (agr),MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Lot Shape_IR1,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,...,Bsmt Exposure_Mn,Bsmt Exposure_No,Garage Finish_Fin,Garage Finish_RFn,Garage Finish_Unf,Lot Frontage,Lot Area,Year Built,Year Remod/Add,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Yr Blt,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,Mo Sold,Yr Sold,Overall Cond,Overall Qual
0,0.0,0.0,0.0,3.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,2.669279,2.541096,-0.537217,-1.159779,0.809950,0.670462,-0.368129,-0.081646,0.157076,1.258181,-0.842670,-0.117647,0.493615,1.151483,-0.252033,-1.027792,-0.767143,0.197861,-0.141622,0.472589,1.782616,-0.720307,0.258970,0.285649,1.182051,0.757070,-0.430381,-0.113091,-0.309413,-0.066758,-0.190872,-0.414845,1.616676,-0.452481,-0.022186
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.525415,0.577464,-0.507706,-1.128207,-0.767414,0.317560,2.720129,-0.617002,-0.323049,-0.657432,-0.842670,-0.117647,-1.451482,-0.842074,-0.252033,-1.027792,-0.767143,-1.040050,-0.141622,-0.956186,-0.988435,-0.684784,-1.007225,1.183096,0.812888,-0.993466,-0.430381,-0.113091,3.231985,-0.066758,-0.190872,-0.033869,1.616676,0.482078,-0.765380
2,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.566562,1.012291,-0.595214,-1.221698,0.778855,1.131166,-0.368129,-0.183596,0.693822,0.617919,-0.842670,-0.117647,-0.207813,-0.842074,-0.252033,-1.027792,1.288899,0.197861,-0.141622,-0.177517,-0.988435,-0.790804,-1.007225,-0.740079,1.728307,0.243186,-0.430381,-0.113091,-0.309413,-0.066758,5.243946,-0.033869,1.616676,0.482078,-0.022186
3,3.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,1.037903,0.488387,-0.290255,-0.894259,-0.767414,1.324253,-0.368129,1.145152,2.121218,1.952530,-0.842670,-0.117647,1.212083,1.151483,-0.252033,0.791276,1.288899,0.197861,-0.141622,1.017913,1.782616,-0.430446,0.258970,0.258217,-0.897997,-0.993466,-0.430381,-0.113091,-0.309413,-0.066758,-0.190872,-0.808867,1.616676,-0.452481,0.667296
4,0.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.270930,0.947835,0.864076,0.582298,-0.767414,0.931750,-0.368129,-1.095273,-0.207085,-0.540763,1.083004,-0.117647,0.443468,-0.842074,-0.252033,0.791276,1.288899,0.197861,-0.141622,-0.177517,0.767561,0.766894,0.258970,0.073811,1.190581,0.191563,-0.430381,-0.113091,-0.309413,-0.066758,-0.190872,-1.213567,1.616676,-0.452481,-0.765380
